# Purpose
This notebook is written as a requirement for the lab [AI Language Technology](https://www.researchgate.net/project/AI-Language-Technology) (WiSe 2018) offered by the Fraunhofer IAIS. It details our implementation of the method to disambiguate word sense in NLP published in the following paper - 

Chen, X., Liu, Z., & Sun, M. (2014). A unified model for word sense representation and disambiguation. In Proceedings of the 2014 Conference on Empirical Methods in Natural Language Processing (EMNLP) (pp. 1025-1035). Accessed at https://www.aclweb.org/anthology/D14-1110.

**Note:** For the sake of explanation, at some places below, we have copied texts in part from the referred research paper. If a text was copied as is, it is quoted.

# Scope
The paper describes the method of disambiguating word sense as a **3-stage** process,
1. Initializing word vectors and sense vectors
2. Performing word sense disambiguation
3. Learning sense vectors from relevant occurrences

Our scope of this lab is to only perform the disambiguation of word sense. Hence, we do not go into the 3rd stage.

# Implementation
A code draft of the algorithm implementation can be found [here](https://github.com/iftekhar-ahmed/wsd-nlp).
A code draft of the experimental setting can be found [here](https://gist.github.com/logicalforhad/2baeff727fb3a711297de1dc83369d7c).

We use 2 python packages - `nltk` & `numpy`. From `nltk` we use the wordnet, punkt & averaged_perceptron_tagger libraries. The last one is the parts of speech (pos) tagger. 

```python
import nltk
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet as wn
import numpy as np
from numpy import dot
from numpy import average
from numpy.linalg import norm
```
## Stage 1: Initializing word vectors and sense vectors
There are 2 parts to this stage.
### Initializing word vectors
The paper uses [Skipgram](https://becominghuman.ai/how-does-word2vecs-skip-gram-work-f92e0525def4) to train the word vectors from large amounts of text data. Since we use [glove](https://nlp.stanford.edu/projects/glove/) for word vector representation, we skip the training. Below is a method to load the word vectors.
```python
def load_glove_vectors(glove_file):
    f = open(glove_file, 'r')
    vectors = {}
    for line in f:
        split_line = line.split()
        word = split_line[0]
        embedding = np.array([float(val) for val in split_line[1:]])
        vectors[word] = embedding
    f.close()
    return vectors
```
This returns a dictionary of words and their corresponding vector embedding.

(Our glove data can be found [here](https://nlp.stanford.edu/projects/glove/))
### Initializing sense vectors
As outlined in Initializing sense vectors in Section 2.2 of the paper, we only consider words that are either *noun*, *verb*, *adjective* or *adverb* as candidates for disambiguation.

Each candidate word has one or more *sense* and each sense has one or more *gloss* in wordnet. For each sense, the task is to find its cosine similarities to each of the words in its gloss; then only take those words that clear a similarity threshold.
```python
cosine_sim_threshold = 0.05 # can be experimented with values ranging between 0-1
```
We find the normalized dot product between a sense vector and a word vector in its gloss to find their cosine similarity.
```python
cos_sim = dot(gloss_word_vec, candidate_vec) / (norm(gloss_word_vec) * norm(candidate_vec))
if cos_sim > cosine_sim_threshold:
    word_vectors.append(gloss_word_vec)
```
Finally, the average of the vectors for these words is used as the initialization value of the sense vector.
```python
sense_vector = average(word_vectors)
```
The method below returns all sense vectors for a candidate word -
```python
def get_word_sense_vectors(candidate):
    vectors = {}
    try:
        candidate_vec = glove[candidate]
    except Exception:
        print(candidate, "not found in glove")
        return None
    for ss in wn.synsets(candidate):
        print("synonym of ", candidate, " is ", ss.lemmas()[0].name())
        tokens = nltk.word_tokenize(ss.definition())
        pos_tags = nltk.pos_tag(tokens)
        word_vectors = []
        for gloss_pos, tag in pos_tags:
            if tag == 'VBD' or tag == 'NN' or tag == 'ADJ' or tag == 'ADV':
                try:
                    gloss_word_vec = glove[gloss_pos]
                except Exception:
                    print(gloss_pos, "not found in glove")
                    continue
                cos_sim = dot(gloss_word_vec, candidate_vec) / (norm(gloss_word_vec) * norm(candidate_vec))
                if cos_sim > cosine_sim_threshold:
                    word_vectors.append(gloss_word_vec)
        if len(word_vectors) == 0:
            continue
        sense_vector = average(word_vectors)
        vectors[ss] = sense_vector
    return vectors
```
## Stage 2: Performing Word Sense Disambiguation
In order to disambiguate all of the content words in a sentence, the authors of the paper designed 2 algorithms namely - L2R (left to right) & S2C (simple to complex). They describe their algorithms as followed -
> The main difference between L2R and S2C is the order of words when performing word sense disambiguation. When given a sentence, the L2R algorithm disambiguates the words from left to right (the natural order of a sentence), whereas the S2C algorithm disambiguates the words with fewer senses first. The main idea of S2C algorithm is that the words with fewer senses are easier to disambiguate, and the disambiguation result can be helpful to sambiguate the words with more senses.

For our implementation we chose S2C. There are 3 steps to this.
### Context vector initialization
Similar to the initialization of sense vectors, the average of all of the content words’ vectors in a sentence as the initialization vector of context.
```python
context_vec = average(list(pos_vectors.values()))
```
Here, `pos_vectors` contain the content word vectors from input sentence.
### Ranking words
According to the algorithm, we rank the content words based on the ascending order of their number of senses.
```python
sorted_sense_vectors_collection = sorted(sorted_sense_vectors_collection.items(), key=lambda x: x[1])
```
Here, `sorted_sense_vectors_collection` contains a dictionary of content words and a list containing their corresponding sense vectors - before sorting. Then we pass it to the `sorted()` numpy method.
### Word sense disambiguation
For each content word in the `sorted_sense_vectors_collection`, we compute the cosine similarities between its sense vectors and the context vector. We choose the sense that yields the maximum cosine similarity as its disambiguation result and then use the sense vector to replace the word vector in `pos_vectors`. Then we repeat the 2nd step to recalculate our context vector.

This entire step is repeated until all content words are disambiguated.
```python
def disambiguate_word_sense(word, context_vector):
    vectors = sense_vectors_collection[word]
    cos_sims = {}
    for sense, sense_vector in vectors.items():
        cos_sim = dot(context_vector, sense_vector) / (norm(context_vector) * norm(sense_vector))
        cos_sims[sense.lemmas()[0].name()] = cos_sim
    sorted_list = sorted(cos_sims.items(), key=lambda x: x[1])
    if len(sorted_list) == 0:
        return None
    nearest_sense_word = sorted_list.pop()[0]
    return nearest_sense_word
```
## Pitfalls
### Inadequate pos tags for content word filtering
We could not compile an exhaustive list of tags for the parts of speech (pos) we are interested in with the pos tagger library in `nltk`. There are many different forms of tags for verb, noun, adjective and adverb as we've explored in the library. For this reason, we went on to choose our own tags for filtering the content words.
### Undefined score margin threshold
In step 3 of Word sense disambiguation, we find the sense vector that has the maximum cosine similarity with the context vector, and use that sense to replace the original word. However, as the paper describes
>We choose the sense that yields the maximum cosine similarity as its disambiguation result. If the score margin between the maximum and the second maximum is larger than the threshold ε, we are confident with the isambiguation result of wi and then use the sense vector to replace the word vector in the context vector. Thus, we obtain a
more accurate context vector for other words that are still yet to be disambiguated.

we did not use any score margin threshold (`ε`). This is because the authors did not mention about a specific value for threshold in relevant examples. And although in **Section 3.5 Parameter Influence** they disclosed that their model achieved best performance when this threshold is 0.1, we could not verify that with our dataset.

# Experiment
## Dataset
Our dataset is taken from the following paper -

Koeling, Rob, Diana McCarthy, and John Carroll. "Domain-specific sense distributions and predominant sense acquisition." Proceedings of the conference on Human Language Technology and Empirical Methods in Natural Language Processing. Association for Computational Linguistics, 2005. Accessed at https://www.researchgate.net/publication/220816751_Domain-Specific_Sense_Distributions_and_Predominant_Sense_Acquisition.

Dataset can be accessed [here](http://www.dianamccarthy.co.uk/downloads/hlt2005releasev2.tgz).

## Results
We ran experiment on 41 different words in 3 different categories - banking and commerce, sports & finance from the dataset. For each of the 41 words our dataset contained around 100 examples in each categories. We tried **4** different word embeddings. A summary of our results is given below:

* total __11427__ valid examples were executed for word sense disambiguation
* total __683__ invalid examples identified in gold standard clean dataset
* overall __12110__ examples were executed and dumped to output file

For each of the 4 word embeddings we used, the number of correctly disambiguated senses and the recall value (calculated by dividing the number of correctly disambiguated word senses by the total number of valid examples in gold standard dataset) are listed below.

| Name                                                 | Correct Disambiguation | Recall    |
|:-----------------------------------------------------|:-----------------------|:----------|
| [Glove](https://tinyurl.com/y4dtgcg7)                | 1616                   | **14.14** |
| [Dependency-Based](https://tinyurl.com/y4czh4j7)     | 1216                   | **10.64** |
| [Bag of Words (k = 2)](https://tinyurl.com/yyv5vkfa) | 1435                   | **12.56** |
| [Bag of Words (k = 5)](https://tinyurl.com/yxw28j4g) | 1506                   | **14.18** |

For each word embedding, the corresponding output dump and disambiguation results are listed below.

| Name                 | Output                                 | Results                                 |
|:---------------------|:---------------------------------------|:----------------------------------------|
| Glove                | [Output](https://tinyurl.com/y5q7phyx) | [Results](https://tinyurl.com/yxchmmrd) |
| Dependency-Based     | Output                                 |                                         |
| Bag of Words (k = 2) | Output                                 |                                         |
| Bag of Words (k = 5) | Output                                 |                                         |
